In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
import torch.nn.functional as F
import random
from sklearn import preprocessing
import torch.nn as nn
import torchvision.datasets as data
import torchvision.transforms as transforms
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

random.seed(777)
torch.manual_seed(777)

In [ ]:
train=pd.read_csv("boston_housing_train.csv")
train.head(3)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,10.23300,0.0,18.10,0.0,0.614,6.185,96.7,2.1705,24.0,666.0,20.2,379.70,18.03,14.6
1,1,0.67191,0.0,8.14,0.0,0.538,5.813,90.3,4.6820,4.0,307.0,21.0,376.88,14.81,16.6
2,2,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1


In [ ]:
x_train=train.iloc[:,1:-1]
y_train=train.iloc[:,[-1]]
x_train.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,10.23300,0.0,18.10,0.0,0.614,6.185,96.7,2.1705,24.0,666.0,20.2,379.70,18.03
1,0.67191,0.0,8.14,0.0,0.538,5.813,90.3,4.6820,4.0,307.0,21.0,376.88,14.81
2,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15


In [ ]:
x_train = torch.FloatTensor(np.array(x_train)).to(device)
y_train = torch.FloatTensor(np.array(y_train)).to(device) 

In [ ]:
test=pd.read_csv("boston_housing_test.csv")
test.head(3)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04
1,1,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53
2,2,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07


In [ ]:
test=test.iloc[:,1:]
test.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04
1,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53
2,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07


In [ ]:
test = torch.FloatTensor(np.array(test)).to(device)

In [ ]:
x_train.shape

torch.Size([339, 13])

In [ ]:
learning_rate = 1e-3
training_epochs = 10000
batch_size = 140
drop_prob = 0.5

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
linear1 = torch.nn.Linear(13,26,bias=True)
linear2 = torch.nn.Linear(26,52,bias=True)
linear3 = torch.nn.Linear(52,104,bias=True)
linear4 = torch.nn.Linear(104,52,bias=True)
linear5 = torch.nn.Linear(52,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)

In [ ]:
loss = torch.nn.MSELoss().to(device) 


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
total_batch = len(data_loader)
model_h = []
error_h = []
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)
  
     
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost
        avg_cost /= total_batch
        model_h.append(model)
        error_h.append(avg_cost)

  if epoch % 1000 == 0 :
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0000 cost = 333.029876709
Epoch: 1000 cost = 4.081452370
Epoch: 2000 cost = 2.503092766
Epoch: 3000 cost = 2.540337801
Epoch: 4000 cost = 1.181686044
Epoch: 5000 cost = 3.455243826
Epoch: 6000 cost = 1.220795274
Epoch: 7000 cost = 0.801764369
Epoch: 8000 cost = 0.443378299
Epoch: 9000 cost = 0.359909236
Epoch: 9999 cost = 0.314769834


In [ ]:
best_model = model_h[np.argmin(error_h)]

In [ ]:
with torch.no_grad():
  model.eval()

  prediction = model(test)
  
prediction = prediction.cpu().numpy().reshape(-1,1)

In [ ]:
submit=pd.read_csv('sample_submission.csv')
submit

,id,Price
0,0.0,1.0
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
...,...,...
162,162.0,1.0
163,163.0,1.0
164,164.0,1.0
165,165.0,1.0


In [ ]:
for i in range(len(prediction)):
  submit['id'][i]=i
  submit['Price'][i]=prediction[i].item()

submit

,id,Price
0,0.0,27.849438
1,1.0,33.503643
2,2.0,15.261737
3,3.0,24.603020
4,4.0,17.870897
...,...,...
162,162.0,20.388874
163,163.0,23.001083
164,164.0,27.756124
165,165.0,18.853836
